**.predict()** : generates output predictions based on the input you pass it (for example, the predicted characters in the MNIST example)

**.evaluate()** : computes the loss based on the input you pass it, along with any other metrics that you requested in the metrics param when you compiled your model (such as accuracy in the MNIST example)

        model.compile(loss='categorical_crossentropy', 
        optimizer=RMSprop(), 
        metrics=['accuracy']) 
        history = model.fit(x_train, y_train, 
        batch_size=batch_size, 
        epochs=epochs, 
        verbose=1, 
        validation_data=(x_test, y_test)) 
        
        predictions = model.predict(x_test) 
        print('First prediction:', predictions[0]) 
        
        score = model.evaluate(x_test, y_test, verbose=0) 
        print('Test loss:', score[0]) 
        print('Test accuracy:', score[1]) 

### COMPARAÇAO ENTRE OS WEIGHTS CRIADOS POR ZHEN ENTRE BATCH_TYPE = { 1 , 2 }

E com tal mudança no agrupando dos frames em relacao ao ultima batch (bt=1 vs 2), observa-se:
        
- maior dificuldade em distinguir corretamente videos normais [ < TN (pred=0/true=0) ]
- aumento das predicoes de anomalia quando o video é normal [ > FP (pred=1/true=0) ]

Uma transicao de True Negativo para Falso Positivo. 

Os casos de Positivo mantiveram-se constantes entre os dois casos.



![Alt text](../zhen++/parameters_results/original_bt/1625759299.9331803_2_4_8_xdviolence_model_weights_CM1.png)
![Alt text](../zhen++/parameters_results/original_bt/1625759299.9331803_2_4_8_xdviolence_model_weights_CM2.png)

***

![Alt text](../zhen++/parameters_results/original_bt/1626306295.6228774_2_4_8_xdviolence_model_weights_CM1.png)![Alt text](../zhen++/parameters_results/original_bt/1626306295.6228774_2_4_8_xdviolence_model_weights_CM2.png)

***

![Alt text](../zhen++/parameters_results/original_bt/1626691755.4069657_2_4_8_xdviolence_model_weights_CM1.png)![Alt text](../zhen++/parameters_results/original_bt/1626691755.4069657_2_4_8_xdviolence_model_weights_CM2.png)

***

![Alt text](../zhen++/parameters_results/original_bt/1626947956.798592_2_4_8_xdviolence_model_weights_CM1.png)![Alt text](../zhen++/parameters_results/original_bt/1626947956.798592_2_4_8_xdviolence_model_weights_CM2.png)

***

![Alt text](../zhen++/parameters_results/original_bt/1627169149.3222094_2_4_8_xdviolence_model_weights_CM1.png)![Alt text](../zhen++/parameters_results/original_bt/1627169149.3222094_2_4_8_xdviolence_model_weights_CM2.png)

***

![Alt text](../zhen++/parameters_results/original_bt/1627553113.262731_2_4_8_xdviolence_model_weights_CM1.png)![Alt text](../zhen++/parameters_results/original_bt/1627553113.262731_2_4_8_xdviolence_model_weights_CM2.png)

***

![Alt text](../zhen++/parameters_results/original_bt/1627900597.7432737_2_4_8_xdviolence_model_weights_CM1.png)![Alt text](../zhen++/parameters_results/original_bt/1627900597.7432737_2_4_8_xdviolence_model_weights_CM2.png)

### SAVE MODEL FORMATS W/ CUSTOM ACTIVATIONS FX EXPERIMENTS

https://www.tensorflow.org/guide/keras/save_and_serialize

https://www.tensorflow.org/guide/saved_model

https://colab.research.google.com/drive/1gfvcXwBDel8USWuMeb-hrSKihXGy_bSl?usp=sharing#scrollTo=PBIrwGo9IXvc

In [1]:
import tensorflow as tf
print(tf.version.VERSION)

import numpy as np
from tensorflow import keras

def gelu(x):
    return 0.5*x*(1+tf.tanh(np.sqrt(2/np.pi)*(x+0.044715*tf.pow(x, 3))))

def get_model():
    # Create a simple model.
    inputs = keras.Input(shape=(32,))
    outputs = keras.layers.Dense(1, activation=gelu)(inputs)
    
    model = keras.Model(inputs, outputs)
    
    model.compile(optimizer="adam", loss="mean_squared_error")
    
    return model


model = get_model()

# Train the model.
test_input = np.random.random((128, 32))
test_target = np.random.random((128, 1))
model.fit(test_input, test_target,epochs=3)

# Calling `save('my_model')` creates a SavedModel folder `my_model`.
model.save("/media/jtstudents/HDD/.zuble/vigia/zu++/modeltf")
model.save("modelh5.h5")


#json_config = model.to_json()
#new_model = keras.models.model_from_json(json_config,custom_objects={"gelu":gelu})
#new_model.predict(test_input)[0]


2.2.0
Epoch 1/3
4/4 [==============================] - 0s 1ms/step - loss: 0.1759
Epoch 2/3
4/4 [==============================] - 0s 1ms/step - loss: 0.1647
Epoch 3/3
4/4 [==============================] - 0s 2ms/step - loss: 0.1568
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /media/jtstudents/HDD/.zuble/vigia/zu++/modeltf/assets


In [2]:
#SavedModel format
modeltf = tf.saved_model.load('/media/jtstudents/HDD/.zuble/vigia/zu++/modeltf')
modeltf = tf.keras.models.load_model('/media/jtstudents/HDD/.zuble/vigia/zu++/modeltf')

modeltf.summary()

for l in modeltf.layers:
    try:
        print(l.activation)
    except: # some layers don't have any activation
        pass

#tf.keras.utils.plot_model(modeltf,show_shapes=True)


#Raises an AssertionError if two objects are not equal up to desired tolerance.
np.testing.assert_allclose(
    model.predict(test_input), modeltf.predict(test_input)
)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 33
Trainable params: 33
Non-trainable params: 0
_________________________________________________________________


In [3]:
#h5 format
modelh5 = tf.keras.models.load_model('/media/jtstudents/HDD/.zuble/vigia/zu++/modelh5.h5',custom_objects={"gelu":gelu})
modelh5.summary()

for l in modelh5.layers:
    try:
        print(l.activation)
    except: # some layers don't have any activation
        pass

tf.keras.utils.plot_model(modelh5,show_shapes=True)

#Raises an AssertionError if two objects are not equal up to desired tolerance.
np.testing.assert_allclose(
    modeltf.predict(test_input), modelh5.predict(test_input)
)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 33
Trainable params: 33
Non-trainable params: 0
_________________________________________________________________
<function gelu at 0x7f39eb1f0158>


In [ ]:
import h5py
import json

#model_info = h5py.File('/media/jtstudents/HDD/.zuble/vigia/zu++/model/model/1670428387.3704295_3gelu_xdviolence_model.h5', 'r')
model_info = h5py.File('/media/jtstudents/HDD/.zuble/vigia/zu++/modelh5.h5', 'r')

model_config = json.loads(model_info.attrs.get('model_config').decode('utf-8'))

for k in model_config['config']['layers']:
    if 'activation' in k['config']:
        print(f"{k['class_name']}: {k['config']['activation']}")